In [1]:
#calcular el SDI 

In [2]:
import os
import pandas as pd
from datetime import datetime
import calendar
import numpy as np
from scipy.stats import gamma, norm

In [3]:
# Set the working directory
os.chdir("C:/Dinagua/Scripts_R_indice_caudales_mensuales")

In [20]:
# Importing and converting the csv input file
codcuenca = "RN"
filename = f"inputTeste/{codcuenca}_monthly_paso_borracho.csv"
data = pd.read_csv(filename, header=0)
data['Fecha'] = pd.to_datetime(data['Fecha'], format="%d/%m/%Y")
station_name = os.path.splitext(os.path.basename(filename))[0]

In [21]:
# Importing the reference period (scale). Type the Time Scale (1 to 12)
k = 3
# Define the month in which the Hydrological Year starts (1 to 12). In Uruguay, starts in April (4)
m = 4
m = calendar.month_name[m]

In [22]:
# Calcula el mes de inicio (StartMonth)
data['StartMonth'] = (data['Fecha'] - pd.DateOffset(months=k-1)).dt.month.map(lambda x: calendar.month_abbr[x])

# Calcula el mes de fin (EndMonth)
data['EndMonth'] = data['Fecha'].dt.month.map(lambda x: calendar.month_abbr[x])

# Calcula la columna ScaleMonth
data['ScaleMonth'] = data['StartMonth'] + '-' + data['EndMonth']

# Calcula la suma acumulativa (cumCaudal)
data['cumCaudal'] = data['Caudal'].rolling(window=k, min_periods=k, closed='right').sum()

# Calcular el logaritmo natural de cumCaudal (lnCaudal)
data['lnCaudal'] = np.log(data['cumCaudal'])

# Muestra las primeras filas del DataFrame resultante
print(data[['Fecha', 'ScaleMonth', 'cumCaudal', 'lnCaudal']])

         Fecha ScaleMonth   cumCaudal  lnCaudal
0   1980-05-01    Mar-May         NaN       NaN
1   1980-06-01    Apr-Jun         NaN       NaN
2   1980-07-01    May-Jul  472.025590  6.157033
3   1980-08-01    Jun-Aug  631.302596  6.447785
4   1980-09-01    Jul-Sep  372.896029  5.921300
..         ...        ...         ...       ...
517 2023-06-01    Apr-Jun   49.880056  3.909621
518 2023-07-01    May-Jul  202.561789  5.311045
519 2023-08-01    Jun-Aug  234.619233  5.457964
520 2023-09-01    Jul-Sep  609.097767  6.411979
521 2023-10-01    Aug-Oct  527.629767  6.268395

[522 rows x 4 columns]


In [23]:
# Obtén los valores únicos de ScaleMonth
iteration = data['ScaleMonth'].unique()

# Inicializa statsOutput como una matriz de ceros
statsOutput = np.zeros((len(iteration), 4))

# Inicializa un contador de fila
rw = 0

In [24]:
# Inicializar el arreglo para almacenar las estadísticas
stats_output = np.zeros((len(iteration), 4))

# Inicializar el contador de filas
rw = 0

# Paso 6: Calcular el SDI con una distribución Gamma
for i in iteration:
    # Extraer los datos basados en la columna 'ScaleMonth'
    extracted_data = data[data['ScaleMonth'] == i]

    # Calcular métricas (media, desviación estándar) de cumCaudal y cumLnCaudal
    stats_output[rw, 0] = np.mean(extracted_data['cumCaudal'])
    stats_output[rw, 1] = np.std(extracted_data['cumCaudal'])
    stats_output[rw, 2] = np.mean(extracted_data['lnCaudal'])
    stats_output[rw, 3] = np.std(extracted_data['lnCaudal'])

    # Calcular el SDI
    data.loc[data['ScaleMonth'] == i, 'SDI'] = (
        (data.loc[data['ScaleMonth'] == i, 'cumCaudal'] - stats_output[rw, 0]) /
        stats_output[rw, 1]
    )

    # Calcular el SDI con una distribución log-normal de dos parámetros
    data.loc[data['ScaleMonth'] == i, 'log_SDI'] = (
        (data.loc[data['ScaleMonth'] == i, 'lnCaudal'] - stats_output[rw, 2]) /
        stats_output[rw, 3]
    )

    # Ajuste con MLE y especificando que la distribución es 'gamma'
    fit_gamma = gamma.fit(extracted_data['cumCaudal'].dropna(), floc=0, optimizer=None)
    # La función gamma en python tiene un parámetro a más que en R "loc" que debe mantenerse en 0 para que los resultados sean compatibles
    
    # Obtener los parámetros ajustados
    alfa, loc, scale = fit_gamma

    # Imprimir los parámetros
    print(f"Parámetro alfa (forma): {alfa}")
    print(f"Parámetro loc (ubicación): {loc}")
    print(f"Parámetro scale (escala): {scale}")
    
    # Asegurarse de que haya suficientes datos después de eliminar NaN
    if len(extracted_data['cumCaudal'].dropna()) > 0:
        cdf_val = gamma.cdf(extracted_data['cumCaudal'], fit_gamma[0], loc=fit_gamma[1], scale=fit_gamma[2])
        inv_normal = norm.ppf(cdf_val)

        # Asignar los valores a 'Gamma_SDI' sin crear una serie de NaN
        data.loc[data['ScaleMonth'] == i, 'Gamma_SDI'] = inv_normal

    # Incrementar el contador de filas
    rw += 1

# Imprimir los parámetros
print(data)

Parámetro alfa (forma): 0.8638861476870198
Parámetro loc (ubicación): 0
Parámetro scale (escala): 463.0426393912688
Parámetro alfa (forma): 1.0458788180416336
Parámetro loc (ubicación): 0
Parámetro scale (escala): 448.82347008531065
Parámetro alfa (forma): 1.5864936694769844
Parámetro loc (ubicación): 0
Parámetro scale (escala): 300.9833709211475
Parámetro alfa (forma): 2.148538251806995
Parámetro loc (ubicación): 0
Parámetro scale (escala): 200.73842256967356
Parámetro alfa (forma): 2.4465995197228807
Parámetro loc (ubicación): 0
Parámetro scale (escala): 181.36803688795726
Parámetro alfa (forma): 2.219540871910455
Parámetro loc (ubicación): 0
Parámetro scale (escala): 209.67797510435437
Parámetro alfa (forma): 2.265472037669449
Parámetro loc (ubicación): 0
Parámetro scale (escala): 198.09494054729362
Parámetro alfa (forma): 1.4177245251706483
Parámetro loc (ubicación): 0
Parámetro scale (escala): 256.1767021497272
Parámetro alfa (forma): 0.9010300120641161
Parámetro loc (ubicación): 

In [25]:
# Crea un DataFrame a partir de statsOutput
statsOutput = pd.DataFrame(data=statsOutput, columns=["avgCumCaudal", "stdCumCaudal", "avgLnCumCaudal", "stdLnCumCaudal"])

# Agrega la columna ScaleMonth
statsOutput["ScaleMonth"] = iteration

# Calcula el año hidrológico y crea la columna hydroYear en el DataFrame 'data'
data['WYear'] = pd.to_datetime(data['Fecha']).dt.year
data['hydroYear'] = data.apply(lambda row: f"{row['WYear']}-{row['WYear']+1}", axis=1)

# Imprimir las estadísticas
print(data)

         Fecha      Caudal StartMonth EndMonth ScaleMonth   cumCaudal  \
0   1980-05-01   29.812059        Mar      May    Mar-May         NaN   
1   1980-06-01  273.857467        Apr      Jun    Apr-Jun         NaN   
2   1980-07-01  168.356065        May      Jul    May-Jul  472.025590   
3   1980-08-01  189.089065        Jun      Aug    Jun-Aug  631.302596   
4   1980-09-01   15.450900        Jul      Sep    Jul-Sep  372.896029   
..         ...         ...        ...      ...        ...         ...   
517 2023-06-01   19.497467        Apr      Jun    Apr-Jun   49.880056   
518 2023-07-01  159.228000        May      Jul    May-Jul  202.561789   
519 2023-08-01   55.893767        Jun      Aug    Jun-Aug  234.619233   
520 2023-09-01  393.976000        Jul      Sep    Jul-Sep  609.097767   
521 2023-10-01   77.760000        Aug      Oct    Aug-Oct  527.629767   

     lnCaudal       SDI   log_SDI  Gamma_SDI  WYear  hydroYear  
0         NaN       NaN       NaN        NaN   1980  1980-

In [26]:
# Filtra las columnas que no necesitas
dataExport = data.drop(columns=['Caudal', 'StartMonth', 'EndMonth', 'cumCaudal', 'WYear', 'lnCaudal'])

# Reordena las columnas
dataExport = dataExport[['Fecha', 'hydroYear', 'ScaleMonth', 'SDI', 'log_SDI', 'Gamma_SDI']]

# Redondea las columnas específicas
dataExport['SDI'] = dataExport['SDI'].round(2)
dataExport['log_SDI'] = dataExport['log_SDI'].round(2)
dataExport['Gamma_SDI'] = dataExport['Gamma_SDI'].round(2)

# Renombra las columnas
dataExport.columns = ["Fecha", "Año_hidrologico", "Escala", "SDI", "LogSDI", "GammaSDI"]

# Crea el nombre de archivo
SDI_Filename = f"teste/{k}-month_CompleteSDI_octubre_{station_name.split('monthly_')[1]}.txt"

# Exporta a un archivo CSV
dataExport.to_csv(SDI_Filename, index=False, sep=",")